In [1]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as  plt
import plotly.graph_objects as go
import plotly.express as px

In [2]:
df=pd.read_csv('/kaggle/input/re-generation-data-set/RE-generation region wise.csv')

In [3]:
df.head()

,Region_name,DATE,Wind,Solar,Biomass,Bagasse,Small Hydel,Others,RE Generation Total(MU)
0,ER,Apr-2023,0.000000,103.759057,6.004081,11.892387,35.281504,128.709313,285.646342
1,NER,Apr-2023,0.000000,29.174491,0.000000,0.000000,18.694489,0.615000,48.483980
2,NR,Apr-2023,676.559764,4219.971476,114.324134,451.876474,300.850729,55.569264,5819.151840
3,SR,Apr-2023,1264.626000,3914.898643,20.179000,325.315000,107.593400,37.413000,5670.025043
4,WR,Apr-2023,2308.011100,2050.889721,155.109111,211.231500,123.786311,2.808690,4851.836433


## DATA EXTRACTION
*
*
*

In [4]:
import pandas as pd

# Create a new DataFrame for summarizing data by month
df_monthly = pd.DataFrame()

# Extract unique months from the 'DATE' column
months = df['DATE'].unique()

# Iterate over each month
for month in months:
    # Filter data for the current month
    df_month = df[df['DATE'] == month]
    
    # Calculate the total renewable energy generation for each region in the current month
    region_totals = df_month.groupby('Region_name')['RE Generation Total(MU)'].sum()
    
    # Add the region totals to the monthly summary DataFrame
    df_monthly[month] = region_totals

df_monthly=df_monthly.T    
df_monthly.index.name='DATE'
# Display the summarized data by month
df_monthly.head()


Region_name,ER,NER,NR,SR,WR
DATE,,,,,
Apr-2023,285.646342,48.483980,5819.151840,5670.025043,4851.836433
Mar-2023,293.833141,44.651901,5570.353077,6632.881943,4622.996361
Feb-2023,287.616586,37.309284,4982.674147,5925.333381,4366.424531
Jan-2023,284.395461,43.556284,5098.823733,6609.051404,5699.556772
Dec-2022,303.293548,55.603867,4796.754022,5653.680943,4786.091870


In [5]:
# df_region = df.groupby('Region_name')['RE Generation Total(MU)'].sum().reset_index()

# # Display the summarized data by region
# print(df_region)



# Create a new DataFrame for summarizing data by region
df_region = df.groupby('Region_name').agg({
    'RE Generation Total(MU)': 'sum',
    'Wind': 'sum',
    'Solar': 'sum',
    'Biomass': 'sum',
    'Bagasse': 'sum',
    'Small Hydel': 'sum',
    'Others': 'sum'
}).reset_index()

# Calculate the sum of all other rows
other_sum = df[['RE Generation Total(MU)', 'Wind', 'Solar', 'Biomass', 'Bagasse', 'Small Hydel', 'Others']].sum().values.tolist()

# Add the sum of all other rows as a new row in the summarized data by region
df_region.loc[len(df_region)] = ['Other'] + other_sum

# Display the summarized data by region
df_region.head()


,Region_name,RE Generation Total(MU),Wind,Solar,Biomass,Bagasse,Small Hydel,Others
0,ER,9812.894935,54.660000,2723.805075,252.720320,1499.328717,1789.511793,3492.869030
1,NER,1179.110991,0.000000,411.461328,0.000000,5.030000,761.350663,1.269000
2,NR,132306.059765,19479.772248,82351.586917,3870.405050,11890.423376,13664.536648,1049.335526
3,SR,233378.478234,101140.052850,107044.083230,1363.282320,11678.494134,11254.865800,897.699900
4,WR,156587.478900,89398.422754,46103.355525,4430.270763,12313.853440,4215.930468,125.645950


#### New data frame that summarize the data of all sectors including all regions as one in Monthly basis
* that is monthly basis preduction of re in all regions

In [6]:

# Create a new DataFrame for summarizing data by unique dates
df_summary = pd.DataFrame()

# Extract unique dates (months) from the 'DATE' column
dates = df['DATE'].unique()

# Iterate over each date
for date in dates:
    # Filter data for the current date
    df_date = df[df['DATE'] == date]
    
    # Calculate the total values for each column in the current date
    date_summary = df_date[['Wind', 'Solar', 'Biomass', 'Bagasse', 'Small Hydel', 'Others', 'RE Generation Total(MU)']].sum()
    
    # Add the date summary to the new DataFrame
    df_summary[date] = date_summary

# Transpose the summary DataFrame to have dates as rows and columns as entities
df_summary = df_summary.T

# Set the 'DATE' column as the index
df_summary.index.name = 'DATE'

# Display the summarized data by unique dates
df_summary.head(36)
df_summary.to_csv('Total Renewable Energy.csv',index=False)

In [7]:
import pandas as pd

# Create a new DataFrame for summarizing data by region for the specified years
df_region_year = pd.DataFrame()

# Extract the year from the 'DATE' column
df['Year'] = pd.to_datetime(df['DATE'], format='%b-%Y').dt.year

# Drop rows with '2023' and '2019' years
#df_filtered = df[(df['Year'] != 2023) & (df['Year'] != 2019)]

# Group the remaining data by region and year
df_grouped = df.groupby(['Region_name', 'Year']).sum().reset_index()

# Pivot the data to have regions as columns
df_pivot = df_grouped.pivot(index='Year', columns='Region_name', values='RE Generation Total(MU)')

# Calculate the sum of all other columns
df_pivot['RE Generation Total'] = df_pivot.sum(axis=1)

# Sort the columns in the desired order
desired_order = ['ER', 'NER', 'WR', 'SR', 'NR', 'RE Generation Total']
df_pivot = df_pivot.reindex(columns=desired_order)

# Display the summarized data by region for the specified years
df_pivot.head(20)


/tmp/ipykernel_32/1056141685.py:13: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_grouped = df.groupby(['Region_name', 'Year']).sum().reset_index()


Region_name,ER,NER,WR,SR,NR,RE Generation Total
Year,,,,,,
2019,2032.930000,232.720000,30849.940000,52253.910000,19703.980000,105073.480000
2020,437.495000,32.650000,6595.030000,9947.130000,4929.320000,21941.625000
2021,2429.350000,203.340000,38560.700000,60575.090000,27012.426000,128780.906000
2022,3761.628405,536.399543,61040.994803,85765.056463,59189.330968,210293.410182
2023,1151.491530,174.001448,19540.814097,24837.291771,21471.002797,67174.601643
